# **IMT - Instituto Mauá de Tecnologia**
## Pós-Graduação em Ciência de Dados e Inteligência Artificial

# **Big Data - Estudo do Dataset "Mapa da Felicidade"**

<br> Por [Gabriel Lima](http://https://github.com/GabrielLima1995),  [Giovanni Schiffini](https://github.com/schiffini) e Rafael Brayner Costa. <h1>

**INTRODUÇÃO**

Em 2011 a Organização das Nações Unidas (ONU) aprovou uma resolução que reconhece a busca pela felicidade como "um objetivo humano fundamental" e convida os estados-membros a promover políticas públicas que incluam a importância da felicidade e do bem-estar em sua aposta pelo desenvolvimento. Essencialmente, essa é a justificativa de toda e qualquer política pública.

A resolução da ONU recebeu o título "A Felicidade: para um Enfoque Holístico do Desenvolvimento" e reconhece que a felicidade é "um objetivo e uma aspiração universal" e que também é "a manifestação do espírito dos Objetivos de Desenvolvimento do Milênio" (ODM). Os 193 países-membros da ONU foram convidados para empreenderem "a elaboração de novas medidas que reflitam melhor a importância deste objetivo em suas políticas públicas".

O primeiro Relatório de Felicidade Mundial foi lançado em 1º de abril de 2012 como um texto fundamental para a reunião da ONU: Bem-estar e Felicidade: Definindo um Novo Paradigma Econômico, atraindo a atenção internacional. O relatório delineou o estado de felicidade mundial, as causas da felicidade e da miséria e as implicações políticas destacadas por estudos de caso. Em 2013, o segundo Relatório de Felicidade Mundial foi publicado e, desde então, tem sido publicado anualmente, com exceção de 2014. O relatório usa principalmente dados da Pesquisa Mundial Gallup. Cada relatório anual está disponível ao público para download no site World Happiness Report.

**OBJETIVO GERAL**

- Apresentar os melhores países dentro do cada segmento;
- Comparar o Brasil com alguns países da América do Norte e Europa.




**OBJETIVO ESPECÍFICO**

- Apresentar técnicas de processamento em Python voltadas para Big Data, especialmente o Spark.

## PySpark

Instalando o PySpark, que não é um pacote padrão do Google Colab:

In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 43.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=da61e2338990db90a5e3da049b8257c10296f2c31ec900adfb470de389e96d6a
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


Importando as bibliotecas e criando a sessão no Spark:

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.appName("world-happiness").getOrCreate()

Carregando o dataset em CSV:

In [3]:
df = spark.read.csv("2019.csv", inferSchema=True, header=True)


Entendendo o tipo dos dados:

In [4]:
df.printSchema()

root
 |-- Overall rank: integer (nullable = true)
 |-- Country or region: string (nullable = true)
 |-- Score: double (nullable = true)
 |-- GDP per capita: double (nullable = true)
 |-- Social support: double (nullable = true)
 |-- Healthy life expectancy: double (nullable = true)
 |-- Freedom to make life choices: double (nullable = true)
 |-- Generosity: double (nullable = true)
 |-- Perceptions of corruption: double (nullable = true)



Verificando o cabeçalho:

In [10]:
df.show(n=5)

+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|Overall rank|Country or region|Score|GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|
+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|           1|          Finland|7.769|          1.34|         1.587|                  0.986|                       0.596|     0.153|                    0.393|
|           2|          Denmark|  7.6|         1.383|         1.573|                  0.996|                       0.592|     0.252|                     0.41|
|           3|           Norway|7.554|         1.488|         1.582|                  1.028|                       0.603|     0.271|                    0.341|
|           4|          Iceland|7.494|        

Verificando a quantidade total de linhas:

In [11]:
df.count()

156

Descrevendo as colunas:

In [6]:
df.describe().show()

+-------+-----------------+-----------------+-----------------+-------------------+------------------+-----------------------+----------------------------+-------------------+-------------------------+
|summary|     Overall rank|Country or region|            Score|     GDP per capita|    Social support|Healthy life expectancy|Freedom to make life choices|         Generosity|Perceptions of corruption|
+-------+-----------------+-----------------+-----------------+-------------------+------------------+-----------------------+----------------------------+-------------------+-------------------------+
|  count|              156|              156|              156|                156|               156|                    156|                         156|                156|                      156|
|   mean|             78.5|             null|5.407096153846153| 0.9051474358974363|1.2088141025641024|     0.7252435897435898|          0.3925705128205128|0.18484615384615383|      0.110602564

Verificando se há valores do tipo "null":

In [14]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|Overall rank|Country or region|Score|GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|
+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|           0|                0|    0|             0|             0|                      0|                           0|         0|                        0|
+------------+-----------------+-----+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+



Não há nenhuma entrada com valor inválido. Se houvesse, seria necessário criar um novo dataframe sem os valores "null", pois um Dataframe do PySpark é imutável.

Criação do dataframe reduzido somente com os dados que são do nosso interesse:

In [15]:
df_reduzido = df.select(['Country or region', 'Score', 'GDP per capita', 'Healthy life expectancy'])

In [16]:
df_reduzido.show()

+-----------------+-----+--------------+-----------------------+
|Country or region|Score|GDP per capita|Healthy life expectancy|
+-----------------+-----+--------------+-----------------------+
|          Finland|7.769|          1.34|                  0.986|
|          Denmark|  7.6|         1.383|                  0.996|
|           Norway|7.554|         1.488|                  1.028|
|          Iceland|7.494|          1.38|                  1.026|
|      Netherlands|7.488|         1.396|                  0.999|
|      Switzerland| 7.48|         1.452|                  1.052|
|           Sweden|7.343|         1.387|                  1.009|
|      New Zealand|7.307|         1.303|                  1.026|
|           Canada|7.278|         1.365|                  1.039|
|          Austria|7.246|         1.376|                  1.016|
|        Australia|7.228|         1.372|                  1.036|
|       Costa Rica|7.167|         1.034|                  0.963|
|           Israel|7.139|

Removendo os espaços dos nomes das colunas, para facilitar sua utilização em SQL:

In [17]:
df_reduzido=df_reduzido.withColumnRenamed("GDP per capita","gdp_per_capita")

In [18]:
df_reduzido=df_reduzido.withColumnRenamed("Country or Region","country_or_region")

In [20]:
df_reduzido=df_reduzido.withColumnRenamed("Healthy life expectancy","healthy_life_expectancy")

In [22]:
df_reduzido=df_reduzido.withColumnRenamed("Score","score")

Esta função withColumn permite acrescentar, alterar e remover colunas do dataframe.
Abaixo, vemos o resultado das alterações no cabeçalho:

In [27]:
df_reduzido.show(20)

+-----------------+-----+--------------+-----------------------+
|country_or_region|score|gdp_per_capita|healthy_life_expectancy|
+-----------------+-----+--------------+-----------------------+
|          Finland|7.769|          1.34|                  0.986|
|          Denmark|  7.6|         1.383|                  0.996|
|           Norway|7.554|         1.488|                  1.028|
|          Iceland|7.494|          1.38|                  1.026|
|      Netherlands|7.488|         1.396|                  0.999|
|      Switzerland| 7.48|         1.452|                  1.052|
|           Sweden|7.343|         1.387|                  1.009|
|      New Zealand|7.307|         1.303|                  1.026|
|           Canada|7.278|         1.365|                  1.039|
|          Austria|7.246|         1.376|                  1.016|
|        Australia|7.228|         1.372|                  1.036|
|       Costa Rica|7.167|         1.034|                  0.963|
|           Israel|7.139|

Há valores iguais a zero na coluna "GDP per capita". Trata-se de um erro, pois não existe país com PIB igual a zero. Vamos remover estes valores com a função "where" do PySpark:

In [28]:
df_clean = df_reduzido.where("gdp_per_capita>0")

In [29]:
df_clean.show(20)

+-----------------+-----+--------------+-----------------------+
|country_or_region|score|gdp_per_capita|healthy_life_expectancy|
+-----------------+-----+--------------+-----------------------+
|          Finland|7.769|          1.34|                  0.986|
|          Denmark|  7.6|         1.383|                  0.996|
|           Norway|7.554|         1.488|                  1.028|
|          Iceland|7.494|          1.38|                  1.026|
|      Netherlands|7.488|         1.396|                  0.999|
|      Switzerland| 7.48|         1.452|                  1.052|
|           Sweden|7.343|         1.387|                  1.009|
|      New Zealand|7.307|         1.303|                  1.026|
|           Canada|7.278|         1.365|                  1.039|
|          Austria|7.246|         1.376|                  1.016|
|        Australia|7.228|         1.372|                  1.036|
|       Costa Rica|7.167|         1.034|                  0.963|
|           Israel|7.139|

Verificando o conteúdo do novo dataframe, não há valores iguais a zero no "gdp_per_capita":

In [30]:
df_clean.describe().show()

+-------+-----------------+------------------+------------------+-----------------------+
|summary|country_or_region|             score|    gdp_per_capita|healthy_life_expectancy|
+-------+-----------------+------------------+------------------+-----------------------+
|  count|              155|               155|               155|                    155|
|   mean|             null|5.4118645161290315| 0.910987096774194|     0.7281935483870969|
| stddev|             null| 1.115128459651589|0.3929254082593822|     0.2400798594616911|
|    min|      Afghanistan|             2.853|             0.026|                    0.0|
|    max|         Zimbabwe|             7.769|             1.684|                  1.141|
+-------+-----------------+------------------+------------------+-----------------------+



## Spark.SQL

Criamos uma tabela SQL a partir do dataframe "limpo":

In [66]:
df_clean.createOrReplaceTempView('df_clean')

Podemos, portanto, utilizar as ferramentas da linguagem SQL. Começaremos filtrando somente os países nos quais o score de felicidade é maior que 7:

In [34]:
highest_scores = spark.sql("SELECT * FROM df_clean WHERE score > 7")

In [35]:
highest_scores.show()

+-----------------+-----+--------------+-----------------------+
|country_or_region|score|gdp_per_capita|healthy_life_expectancy|
+-----------------+-----+--------------+-----------------------+
|          Finland|7.769|          1.34|                  0.986|
|          Denmark|  7.6|         1.383|                  0.996|
|           Norway|7.554|         1.488|                  1.028|
|          Iceland|7.494|          1.38|                  1.026|
|      Netherlands|7.488|         1.396|                  0.999|
|      Switzerland| 7.48|         1.452|                  1.052|
|           Sweden|7.343|         1.387|                  1.009|
|      New Zealand|7.307|         1.303|                  1.026|
|           Canada|7.278|         1.365|                  1.039|
|          Austria|7.246|         1.376|                  1.016|
|        Australia|7.228|         1.372|                  1.036|
|       Costa Rica|7.167|         1.034|                  0.963|
|           Israel|7.139|

Podemos ordenar nosso dataframe por um valor específico. Segue abaixo uma nova tabela ordenada por PIB per capita. O comando "DESC" é necessário para ordenarmos de forma decrescente, pois o padrão do SQL é em ordem ascendente:

In [37]:
highest_GDP = spark.sql("SELECT * FROM df_clean ORDER BY gdp_per_capita DESC")

In [38]:
highest_GDP.show()

+--------------------+-----+--------------+-----------------------+
|   country_or_region|score|gdp_per_capita|healthy_life_expectancy|
+--------------------+-----+--------------+-----------------------+
|               Qatar|6.374|         1.684|                  0.871|
|          Luxembourg| 7.09|         1.609|                  1.012|
|           Singapore|6.262|         1.572|                  1.141|
|United Arab Emirates|6.825|         1.503|                  0.825|
|              Kuwait|6.021|           1.5|                  0.808|
|             Ireland|7.021|         1.499|                  0.999|
|              Norway|7.554|         1.488|                  1.028|
|         Switzerland| 7.48|         1.452|                  1.052|
|           Hong Kong| 5.43|         1.438|                  1.122|
|       United States|6.892|         1.433|                  0.874|
|        Saudi Arabia|6.375|         1.403|                  0.795|
|         Netherlands|7.488|         1.396|     

Criaremos outra tabela ordenando os valores por expectativa de vida:

In [42]:
highest_health = spark.sql("SELECT * FROM df_clean ORDER BY healthy_life_expectancy DESC")

In [43]:
highest_health.show()

+-----------------+-----+--------------+-----------------------+
|country_or_region|score|gdp_per_capita|healthy_life_expectancy|
+-----------------+-----+--------------+-----------------------+
|        Singapore|6.262|         1.572|                  1.141|
|        Hong Kong| 5.43|         1.438|                  1.122|
|            Japan|5.886|         1.327|                  1.088|
|            Spain|6.354|         1.286|                  1.062|
|      Switzerland| 7.48|         1.452|                  1.052|
|           France|6.592|         1.324|                  1.045|
|           Cyprus|6.046|         1.263|                  1.042|
|  Northern Cyprus|5.718|         1.263|                  1.042|
|           Canada|7.278|         1.365|                  1.039|
|            Italy|6.223|         1.294|                  1.039|
|      South Korea|5.895|         1.301|                  1.036|
|        Australia|7.228|         1.372|                  1.036|
|           Israel|7.139|

Já podemos efetuar diversas análises:
- Os países com maior riqueza por pessoa ou maior expectativa de vida não são, necessariamente, os mais felizes;
- Os países nórdicos (Noruega, Finlândia, Dinamarca, Suécia), a Austrália e o Canadá estão entre os líderes em todos os quesitos;
- Países como Qatar e Singapura são líderes em riqueza e expectativa de vida, mas nem sequer aparecem entre os 20 primeiros na felicidade geral. Isto se deve a outros fatores não explorados aqui, como percepção de corrupção e liberdade individual.

Verificaremos abaixo o valor médio para cada critério analisado:

In [44]:
avg_score = spark.sql("SELECT AVG(score) FROM df_clean;")

In [45]:
avg_gdp = spark.sql("SELECT AVG(gdp_per_capita) FROM df_clean;")

In [46]:
avg_health = spark.sql("SELECT AVG(healthy_life_expectancy) FROM df_clean;")

In [47]:
avg_score.show()

+------------------+
|        avg(score)|
+------------------+
|5.4118645161290315|
+------------------+



In [48]:
avg_gdp.show()

+-------------------+
|avg(gdp_per_capita)|
+-------------------+
|  0.910987096774194|
+-------------------+



In [49]:
avg_health.show()

+----------------------------+
|avg(healthy_life_expectancy)|
+----------------------------+
|          0.7281935483870969|
+----------------------------+



Comparando os dados do Brasil, nota-se que estamos bem acima da média em todos os quesitos verificados:

In [51]:
brazil= spark.sql("SELECT * FROM df_clean WHERE country_or_region IN ('Brazil')").show()

+-----------------+-----+--------------+-----------------------+
|country_or_region|score|gdp_per_capita|healthy_life_expectancy|
+-----------------+-----+--------------+-----------------------+
|           Brazil|  6.3|         1.004|                  0.802|
+-----------------+-----+--------------+-----------------------+



Comparando o Brasil a outros países que possuem população semelhante (+ 200 milhões), verifica-se que nossos indicadores também estão bem acima da média, inclusive não tão distantes dos Estados Unidos da América:

In [ ]:
comparison= spark.sql("SELECT * FROM df_reduzido WHERE Country_or_region IN ('Brazil', 'United States', 'Pakistan', 'Indonesia', 'Nigeria')").show()

+-----------------+-----+--------------+-----------------------+
|Country_or_region|Score|gdp_per_capita|Healthy_life_expectancy|
+-----------------+-----+--------------+-----------------------+
|    United States|6.892|         1.433|                  0.874|
|           Brazil|  6.3|         1.004|                  0.802|
|         Pakistan|5.653|         0.677|                  0.535|
|          Nigeria|5.265|         0.696|                  0.245|
|        Indonesia|5.192|         0.931|                   0.66|
+-----------------+-----+--------------+-----------------------+



Abaixo, uma lista de todos os países que estão abaixo da média no Score geral de felicidade:

In [52]:
below_avg= spark.sql("SELECT * FROM df_clean WHERE score BETWEEN 0 AND 5.407;").show(200)

+--------------------+-----+--------------+-----------------------+
|   country_or_region|score|gdp_per_capita|healthy_life_expectancy|
+--------------------+-----+--------------+-----------------------+
|Bosnia and Herzeg...|5.386|         0.945|                  0.845|
|              Turkey|5.373|         1.183|                  0.808|
|            Malaysia|5.339|         1.221|                  0.828|
|             Belarus|5.323|         1.067|                  0.789|
|              Greece|5.287|         1.181|                  0.999|
|            Mongolia|5.285|         0.948|                  0.667|
|     North Macedonia|5.274|         0.983|                  0.838|
|             Nigeria|5.265|         0.696|                  0.245|
|          Kyrgyzstan|5.261|         0.551|                  0.723|
|        Turkmenistan|5.247|         1.052|                  0.657|
|             Algeria|5.211|         1.002|                  0.785|
|             Morocco|5.208|         0.801|     

## Acrescentando colunas com o PySpark

Para concluir, voltaremos à ferramenta PySpark.
Criaremos um novo dataframe a partir do original e inseriremos uma nova coluna com a razão entre o Score de felicidade e o PIB per capita:

In [75]:
df_final = df.select(['Country or region', 'Score', 'GDP per capita', 'Healthy life expectancy'])

In [76]:
df_final.withColumn('Score per GDP', (df['Score']) / (df['GDP per capita'])).show(200)

+--------------------+-----+--------------+-----------------------+------------------+
|   Country or region|Score|GDP per capita|Healthy life expectancy|     Score per GDP|
+--------------------+-----+--------------+-----------------------+------------------+
|             Finland|7.769|          1.34|                  0.986|  5.79776119402985|
|             Denmark|  7.6|         1.383|                  0.996|  5.49530007230658|
|              Norway|7.554|         1.488|                  1.028| 5.076612903225807|
|             Iceland|7.494|          1.38|                  1.026| 5.430434782608696|
|         Netherlands|7.488|         1.396|                  0.999| 5.363896848137537|
|         Switzerland| 7.48|         1.452|                  1.052| 5.151515151515152|
|              Sweden|7.343|         1.387|                  1.009| 5.294160057678442|
|         New Zealand|7.307|         1.303|                  1.026| 5.607828089025327|
|              Canada|7.278|         1.365|

## Conclusão

Nota-se no dataframe acima, que o Brasil possui um Score alto proporcionalmente ao seu PIB per capita, ficando a frente, inclusive, da maioria dos países desenvolvidos neste critério. 
É possível observar também que países considerados pobres, como Libéria, Madagascar e outros países africanos, tem um Score de felicidade excelente quando comparado ao seu PIB per capita.

Esta foi uma rápida análise do dataset World Happiness de 2019, mostrando os líderes em cada requisito e o posicionamento do Brasil em relação a países de porte semelhante, demonstrando também a utilização básica do PySpark e do Spark SQL.